## KR260 HLS Configurable Fixed Gain with AXI Streaming Interface
This tutorial demonstrates how to accelerate a Python function on the KR260 using PYNQ, and how to interface with a Vitis HLS modules which has a AXI Streaming interface with an AXI Lite interface for Status/Control

### Plot function for use in this notebook
The first code block below defines a function that we will use for plotting data throughout this notebook. Note that the function has a `n_samples` argument so that we can limit the number of samples to plot. Plotting more than a few thousand samples can be very slow and consume a lot of RAM.

### Requirements
Install the following:

```bash
pip install ipympl
pip install ipywidgets
pip install jupyter_bokeh
```

# Start of Demo Application

#### Bokeh Plot Function

### Hardware HLS Gain Block Implementation
##### Load Overlay

In [1]:
from pynq import Overlay
from pynq import allocate
from pynq import MMIO
import pynq.lib.dma
import time
import numpy as np
import math
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook, show

#import Python library from repo:
import sys
sys.path.append('/home/root/jupyter_notebooks/puch/')
import fpga.py.puch
import fpga.lib.timestamp.sw.timestamp_regmap
import fpga.lib.led_reg.sw.led_regmap


output_notebook()

# Load the overlay
overlay = Overlay('/home/root/jupyter_notebooks/kr260_hls_fixed_gain_stream/kr260_hls_fixed_gain_stream.bit')

# Assign blocks to short vars
dma          = overlay.axi_dma_0
gain_module  = overlay.axi_fixed_gain_stream_0
led_module   = overlay.led_reg_0

Loading BokehJS ...

##### Configure the GAIN register Control Register
Want to enable auto restart, so the FPGA block will always be read to receive samples

In [2]:
gain_module.register_map.CTRL.AUTO_RESTART=1
gain_module.register_map.CTRL.AP_START=1
gain_module.register_map.CTRL

Register(AP_START=1, AP_DONE=0, AP_IDLE=0, AP_READY=0, RESERVED_1=0, AUTO_RESTART=1, RESERVED_2=0, INTERRUPT=0, RESERVED_3=0)

##### Configure the GAIN register Value Register

In [3]:
gain_module.write(gain_module.register_map.gain.address,1)
print("Gain Register : " + str(gain_module.read(gain_module.register_map.gain.address)))

Gain Register : 1


### DMA Transfer the Sinewave buffer from ARM Processor to the FPGA HLS Gain Block, and plot

In [6]:
dma_block_size = 2**10;
iter_num = 2**10
rng = np.random.default_rng()
block_data = rng.integers(low=0, high=2^16, size=dma_block_size)
returned_data = rng.integers(low=0, high=2^16, size=dma_block_size)

# Read Gain Register
print("Gain Register                : " + str(gain_module.read(gain_module.register_map.gain.address)))
print("DMA Transfer Size            : " + str(len(block_data)))

# Allocate buffers for the input and output signals
in_buffer = allocate(shape=(dma_block_size,), dtype=np.int32)
out_buffer = allocate(shape=(dma_block_size,), dtype=np.int32)

good_blocks = 0
bad_blocks = 0
time_log = []
p = figure(title = "Block Transfer Rates", x_axis_label = 'Block Index', y_axis_label = 'Transfer Rate (sec)')
for i in range(iter_num):
    # Copy the samples to the in_buffer
    block_data = rng.integers(low=0, high=2^16, size=dma_block_size)
    
    start_time = time.time()
    np.copyto(in_buffer,block_data)
    
    dma.sendchannel.transfer(in_buffer)
    dma.recvchannel.transfer(out_buffer)
    dma.sendchannel.wait()
    dma.recvchannel.wait()
    
    stop_time = time.time()
    np.copyto(returned_data,out_buffer)
    
    hw_exec_time = stop_time-start_time
    time_log.append(hw_exec_time)
    
    if(np.array_equal(block_data,returned_data)):
        good_blocks += 1
    else:
        bad_blocks += 1
        

print("Min Value                    : "+str(min(time_log)/2**3)+" ms")
print("Max Value                    : "+str(max(time_log)/2**3)+" ms")
print("Min Tranfer Rate             : "+str(32*dma_block_size/max(time_log)/1e6)+" Mbps")
print("Max Tranfer Rate             : "+str(32*dma_block_size/min(time_log)/1e6)+" Mbps")
print("Min Tranfer Rate             : "+str(32*dma_block_size/max(time_log)/1e6/32)+" MSPS")
print("Max Tranfer Rate             : "+str(32*dma_block_size/min(time_log)/1e6/32)+" MSPS")
print("Number of Bad Blocks         : "+str(bad_blocks))
print("Number of Good Blocks        : "+str(good_blocks))
print("Total Bit Transferred        : "+str((bad_blocks+good_blocks)*dma_block_size*32))

p = figure(title = "Block Transfer Rates", x_axis_label = 'Block Index', y_axis_label = 'Transfer Rate (sec)')
x=np.arange(0,len(time_log))
p.line(x,time_log,line_color="blue",line_width=2)
show(p)


# Free the buffers
in_buffer.close()
out_buffer.close()

Gain Register                : 1
DMA Transfer Size            : 1024
Min Value                    : 3.4737586975097657e-06 ms
Max Value                    : 1.3709068298339844e-05 ms
Min Tranfer Rate             : 23.90242669078261 Mbps
Max Tranfer Rate             : 94.33009847083046 Mbps
Min Tranfer Rate             : 0.7469508340869566 MSPS
Max Tranfer Rate             : 2.947815577213452 MSPS
Number of Bad Blocks         : 0
Number of Good Blocks        : 1024
Total Bit Transferred        : 33554432


In [ ]:
output_notebook()
p = figure(title = "Block Transfer Rates", x_axis_label = 'Block Index', y_axis_label = 'Transfer Rate (sec)')
x=np.arange(0,len(time_log))
p.line(x,time_log,line_color="blue",line_width=2)
show(p)